<a href="https://colab.research.google.com/github/FaithoverCP/SHURE/blob/main/Directional_AI_Arena_Alpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Directional AI Arena Alpha
This notebook showcases a minimal alpha version of the Directional AI Arena environment.

## Features
- Loads a base Language Model (GPT-2 by default, easily swappable)
- Applies a *Directional AI Strategic Layer* to transform output
- Inserts *IP trace metadata* for IP tracking (Invisa.ai™, SHURE®, IPIC)
- Serves predictions via a *FastAPI* app with a public URL using *ngrok*

## Next Steps
1. **Run each cell in order.**
2. *Replace the base model* if you have a private or specialized one.
3. Connect to your front-end/UI or incorporate advanced logic for *enhanced reasoning signature*.
4. Consider submission packages (LMSYS) or other expansions.

Let's get started!

In [38]:
%%capture
# 1) Install necessary packages
!pip install fastapi nest-asyncio pyngrok uvicorn transformers torch


In [54]:
# 2) Import libraries
import os
import nest_asyncio
from pyngrok import ngrok
from fastapi import FastAPI, Body
from pydantic import BaseModel
import uvicorn

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# 3) Allow nested event loops for Colab
nest_asyncio.apply()

# 4) Set your ngrok auth token (REQUIRED)
NGROK_AUTH_TOKEN = "2uvVHqwJ7YY9U5PAgdomRCeVnzG_3mfBLKP5DLkdYPfT4WAEK"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# 5) Kill any running ngrok tunnels before starting a new one (Optional but recommended)
ngrok.kill()

# 6) Create a new public ngrok tunnel on port 8000
public_url = ngrok.connect(8000)
print("Ngrok tunnel URL:", public_url)



Ngrok tunnel URL: NgrokTunnel: "https://16c8b7159862.ngrok.app" -> "http://localhost:8000"


In [41]:
# 6) Load base model and tokenizer.
#    By default, we use 'gpt2'. Replace with 'gpt2-medium', 'mistralai/Mistral-7B-v0.1', etc.

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

MODEL_NAME = "gpt2"  # Replace with your preferred model

# Detect if GPU is available and set the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load tokenizer and model
print(f"Loading model: {MODEL_NAME} ...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

# Create generation pipeline explicitly on the chosen device
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if device.type == 'cuda' else -1)

print("✅ Model loaded successfully.")


Using device: cpu
Loading model: gpt2 ...


Device set to use cpu


✅ Model loaded successfully.


## 7) Directional AI Strategic Response Layer
Here, we define a simple function to demonstrate how you can wrap or post-process the model’s output to align with **Directional AI** requirements.

In a real production environment, this could involve specialized ranking, weighting, or dynamic re-routing of sub-prompts. For now, we’ll just do a placeholder function.

In [42]:
def directional_ai_strategic_layer(raw_text: str) -> str:
    """Apply a simple transformation or marker to highlight the Directional AI strategic layer."""
    # Example: add a signature or transform the text in some way.
    # For demonstration, we'll just add a prefix/suffix.
    # In a real scenario, you might do more advanced logic.
    prefix = "[D-AI Strategic Output]\n"
    suffix = "\n[End of D-AI Layer]"
    return f"{prefix}{raw_text.strip()}{suffix}"

# Example usage:
test_str = directional_ai_strategic_layer("This is a test of the Directional AI strategic layer.")
print(test_str)


[D-AI Strategic Output]
This is a test of the Directional AI strategic layer.
[End of D-AI Layer]


## 8) IP Trace Metadata Injection
We can embed relevant IP tracking data (Invisa.ai™, SHURE®, IPIC) in the response.

For demonstration, we’ll attach a dictionary of metadata. In a real production setting, you’d likely store or transmit this metadata separately or embed it in a structured format like JSON.

In [43]:
IP_TRACE_METADATA = {
    "Invisa.ai": "Proprietary Invisa.ai Markers",
    "SHURE": "Strategic High-Utilization Reasoning Engine",
    "IPIC": "Intellectual Property Innovation Coin"
}

def inject_ip_metadata(text: str) -> str:
    """Append IP trace metadata to the output text."""
    metadata_str = "\n\n[IP TRACE METADATA]\n" + "\n".join(
        f"- {k}: {v}" for k, v in IP_TRACE_METADATA.items()
    )
    return text + metadata_str

# Example usage:
sample_text = "Sample output from the model."
print(inject_ip_metadata(sample_text))


Sample output from the model.

[IP TRACE METADATA]
- Invisa.ai: Proprietary Invisa.ai Markers
- SHURE: Strategic High-Utilization Reasoning Engine
- IPIC: Intellectual Property Innovation Coin


## 9) FastAPI + ngrok Setup
Below is a minimal FastAPI app with one POST endpoint `/predict`. You can send a JSON body containing a `prompt` and an optional `max_new_tokens` to generate text.

### Example Request
```bash
curl -X POST \
  -H "Content-Type: application/json" \
  -d '{"prompt":"Hello, world!"}' \
  <YOUR_NGROK_URL>/predict
```
### Response
```json
{
  "output": "[D-AI Strategic Output]\n...\n[End of D-AI Layer]\n\n[IP TRACE METADATA]\n- Invisa.ai: ..."
}
```

In [45]:
app = FastAPI()

class PromptRequest(BaseModel):
    prompt: str
    max_new_tokens: int = 50

@app.post("/predict")
def predict(request: PromptRequest):
    # Generate raw model output
    generated = generator(
        request.prompt,
        max_new_tokens=request.max_new_tokens,
        do_sample=True,
        top_k=50,
        top_p=0.95
    )
    raw_text = generated[0]['generated_text']

    # Pass through Directional AI strategic layer
    strategic_text = directional_ai_strategic_layer(raw_text)

    # Append IP trace metadata
    final_text = inject_ip_metadata(strategic_text)

    return {"output": final_text}


In [46]:
# 10) Launch the FastAPI server asynchronously

print("Starting FastAPI server... please wait.")

def run_app():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Run the server in the background (threaded)
import threading
server_thread = threading.Thread(target=run_app, daemon=True)
server_thread.start()

print("🚀 FastAPI server is running successfully!")
print(f"🌐 Public URL: {public_url.public_url}/predict")

print("\n✅ Example cURL command to test:")
print(f"""curl -X POST -H 'Content-Type: application/json' \\
  -d '{{"prompt":"Hello, Directional AI!", "max_new_tokens":50}}' \\
  {public_url.public_url}/predict""")


Starting FastAPI server... please wait.
🚀 FastAPI server is running successfully!
🌐 Public URL: https://1d26174f3b8f.ngrok.app/predict

✅ Example cURL command to test:
curl -X POST -H 'Content-Type: application/json' \
  -d '{"prompt":"Hello, Directional AI!", "max_new_tokens":50}' \
  https://1d26174f3b8f.ngrok.app/predict


INFO:     Started server process [787]
INFO:     Waiting for application startup.


## You're All Set!
1. **Run all cells** above.
2. Your public *ngrok* URL will be displayed at the bottom.
3. Use it for testing from anywhere or hooking up to your front-end.

### Next Steps
- **LMSYS Submission**: Once tested, let me know if you want the structured submission package.
- **UI Layer**: We can build a Streamlit, Gradio, or custom front-end.
- **Enhanced Reasoning Signature**: Integrate more advanced logic in the `directional_ai_strategic_layer()`.

Let's put **Directional AI** on the map—time to lead the Arena!